In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(SingleR)
    library(celldex)
    library(ShinyCell)
})

In [6]:
# Load Seurat package
library(Seurat)

# URL of the compressed file
url <- "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE179nnn/GSE179487/suppl/GSE179487_FINAL_master_list_of_gene_counts_MIN.Fastqs.txt.gz"

# Download the compressed file to a temporary location
temp_compressed <- tempfile(fileext = ".gz")
download.file(url, temp_compressed)

# Decompress the file
temp_decompressed <- tempfile()
gzfile <- gzfile(temp_compressed, "rb")
writeLines(readLines(gzfile), temp_decompressed)
close(gzfile)

# Read the data into R
data <- read.table(temp_decompressed, header = TRUE)

Warning message:
“Data is of class data.frame. Coercing to dgCMatrix.”
Warning message in matrix(data = as.numeric(x = x), ncol = nc):
“NAs introduced by coercion”


In [56]:
data

id,X19616.021_HiHi_oW_S17,X19616.022_ABC_oW_S20,X19616.022_HiHi_oW_S21,X19616.022_naiveB_oW_S19,X19616.023_HiHi_bL_S15,X19616.023_HiHi_oW_S18,X19616.025_ABC_oW_S27,X19616.025_HiHi_oW_S29,X19616.025_naiveB_oW_S26,⋯,FS1819.131_HiHi_oW_S10,FS1819.131_naiveB_bL_S1,FS1819.131_naiveB_oW_S8,FS1819.131_PB_oW_S9,FS1819.999_ABC_bL_S5,FS1819.999_HiHi_bL_S7,FS1819.999_naiveB_bL_S4,FS1819.999_PB_bL_S6,geneCoordinate,geneSymbol
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
ENSG00000000003,0,0,0,0,0,1,0,2,0,⋯,17,1,3,1,0,1,0,0,chrX:100627009-100640091,TSPAN6
ENSG00000000419,191,113,181,87,221,163,117,142,86,⋯,99,100,91,85,92,179,94,137,chr20:50934767-50958655,DPM1
ENSG00000000457,8,21,21,8,52,24,38,18,19,⋯,10,14,16,26,53,19,13,14,chr1:169849531-169894367,SCYL3
ENSG00000000460,5,1,3,4,16,7,8,5,0,⋯,0,2,4,0,4,2,0,13,chr1:169661907-169854180,C1orf112
ENSG00000000938,125,299,51,457,57,0,794,476,78,⋯,722,433,492,695,233,303,338,495,chr1:27611964-27635377,FGR
ENSG00000000971,0,0,2,0,0,3,0,1,0,⋯,0,1,0,6,0,1,0,0,chr1:196651778-196747604,CFH
ENSG00000001036,20,97,14,58,33,45,99,62,13,⋯,95,54,81,103,36,45,38,76,chr6:143494711-143511790,FUCA2
ENSG00000001084,22,18,21,9,32,50,11,15,6,⋯,20,12,10,26,65,14,13,16,chr6:53497241-53617070,GCLC
ENSG00000001167,85,54,188,100,174,174,56,87,67,⋯,52,74,65,41,36,111,134,52,chr6:41072845-41100076,NFYA


In [52]:
seurat_obj <- suppressWarnings(CreateSeuratObject(counts = data))
rownames(seurat_obj@meta.data) <- sub("^X", "", rownames(seurat_obj@meta.data))

seurat_obj@meta.data$id <- factor(rownames(seurat_obj@meta.data))
seurat_obj@meta.data <- seurat_obj@meta.data[complete.cases(seurat_obj@meta.data), ]

AddMetaData(seurat_obj, metadata = "metadata.txt", col.name = "id")

# head(seurat_obj@meta.data)

ERROR: Error in validObject(object = x): invalid class “Seurat” object: 1: all cells in assays must be present in the Seurat object
invalid class “Seurat” object: 2: 'active.idents' must be as long as the number of cells present
